In [1]:
import torch
import torch.nn as nn
import torch.nn.init
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# convolutional layer

In [2]:
conv = nn.Conv2d(1,1,11,stride = 4, padding = 0)
print(conv)

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))


In [3]:
#batch size * channel * width * height
inputs = torch.Tensor(1,1,227,227)

In [4]:
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 55, 55])


In [5]:
conv = nn.Conv2d(1,1,7,stride = 2, padding = 0)

inputs = torch.Tensor(1,1,64,64)

out = conv(inputs)

print(out.shape)

torch.Size([1, 1, 29, 29])


In [6]:
conv = nn.Conv2d(1,1,5,stride = 1, padding = 2)

inputs = torch.Tensor(1,1,32,32)

out = conv(inputs)

print(out.shape)

torch.Size([1, 1, 32, 32])


In [7]:
inputs = torch.Tensor(1,1,32,64)

conv = nn.Conv2d(1,1,5)

out = conv(inputs)

print(out.shape)

torch.Size([1, 1, 28, 60])


In [8]:
inputs = torch.Tensor(1,1,64,32)

conv = nn.Conv2d(1,1,3,1,1)

out = conv(inputs)

print(out.shape)

torch.Size([1, 1, 64, 32])


In [9]:
#maxpooling

inputs = torch.Tensor(1,1,28,28)

conv1 = nn.Conv2d(1,5,5)

pool = nn.MaxPool2d(2)

out = conv1(inputs)

print(out.shape)

torch.Size([1, 5, 24, 24])


In [10]:
out2 = pool(out)

print(out2.size())

torch.Size([1, 5, 12, 12])


# convolutional neural network

In [11]:
#cuda를 사용가능하면 cuda쓰고 그렇지 않으면 cpu로 연산

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':

    torch.cuda.manual_seed_all(777)

In [12]:
learning_rate = 0.001

training_epochs = 15

batch_size = 100

In [13]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             train = True,
                             transform = transforms.ToTensor(),
                             download = True)

mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            train = False,
                            transform = transforms.ToTensor(),
                            download = True)

100%|██████████| 9912422/9912422 [00:00<00:00, 352988667.57it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 40910399.81it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 145907960.01it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21453298.16it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [14]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [15]:
class CNN(nn.Module):

    def __init__(self):

        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Linear(7*7*64,10,bias=True)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):

        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

In [16]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [17]:
#training

total_batch = len(data_loader)

print('learning started. it takes sometime.')

for epoch in range(training_epochs):

    avg_cost = 0

    for x,y in data_loader:

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('learning finished!')

learning started. it takes sometime.
[epoch:    1] cost = 0.225768194
[epoch:    2] cost = 0.0628905743
[epoch:    3] cost = 0.0459176786
[epoch:    4] cost = 0.0370514505
[epoch:    5] cost = 0.030701058
[epoch:    6] cost = 0.0256532989
[epoch:    7] cost = 0.0215865672
[epoch:    8] cost = 0.0178725924
[epoch:    9] cost = 0.0158482194
[epoch:   10] cost = 0.0131860338
[epoch:   11] cost = 0.00994244777
[epoch:   12] cost = 0.00945247989
[epoch:   13] cost = 0.00831384677
[epoch:   14] cost = 0.00686546415
[epoch:   15] cost = 0.00635658158
learning finished!


In [18]:
#test

with torch.no_grad():

    x_test = mnist_test.test_data.view(len(mnist_test), 1,28,28).float().to(device)
    y_test = mnist_test.test_labels.to(device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()

    print('accuracy:', accuracy.item())

accuracy: 0.9874999523162842


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# 3-layer convolutional neural network

In [21]:
class CNN(nn.Module):

    def __init__(self):

        super(CNN, self).__init__()

        self.keep_prob = 0.5

        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2,1)
        )

        self.fc1 = nn.Linear(4*4*128,625,bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p = 1 - self.keep_prob)
        )

        self.fc2 = nn.Linear(625,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):

        out = self.layer1(x)

        out = self.layer2(out)

        out = self.layer3(out)

        out = out.view(out.size(0), -1)

        out = self.layer4(out)

        out = self.fc2(out)

        return out

In [22]:
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [23]:
#training

total_batch = len(data_loader)

model.train()

print('learning started. it takes sometimes.')

for epoch in range(training_epochs):

    avg_cost = 0

    for x,y in data_loader:

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('learning finished')

learning started. it takes sometimes.
[epoch:    1] cost = 0.190349147
[epoch:    2] cost = 0.0532114543
[epoch:    3] cost = 0.0345323943
[epoch:    4] cost = 0.0288517717
[epoch:    5] cost = 0.0227231309
[epoch:    6] cost = 0.018948324
[epoch:    7] cost = 0.0155169722
[epoch:    8] cost = 0.0133096781
[epoch:    9] cost = 0.0121802473
[epoch:   10] cost = 0.011527976
[epoch:   11] cost = 0.00939154811
[epoch:   12] cost = 0.00802323502
[epoch:   13] cost = 0.00867515523
[epoch:   14] cost = 0.0071749799
[epoch:   15] cost = 0.00773326587
learning finished


In [24]:
#test

with torch.no_grad():

    model.eval()

    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()

    print('accuracy:', accuracy.item())

accuracy: 0.9752999544143677


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
